In [11]:
# Install Libraries

%pip install dash
%pip install pandas
%pip install plotly



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import Libraries

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

In [13]:
# import CSV File

import ssl
from urllib.request import urlopen

# Create an SSL context with certificate verification disabled
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# Open the URL using the SSL context
with urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv', context=ssl_context) as response:
    # Read the CSV data
    data = pd.read_csv(response)

In [14]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Task 2.1
# Set the title of the dashboard

app.layout = html.Div(
    html.H1("Automobile Sales Statistics Dashboard", style={'textAlign': 'center', 'color': '#503D36', 'fontSize': 24})
)

##if __name__ == '__main__':
  ##  app.run_server(debug=True)


In [15]:
# List of years 
year_list = [i for i in range(1980, 2024, 1)]

In [16]:
app.layout = html.Div([
    # TASK 2.1 Add title to the dashboard
    html.H1("Automobile Sales Statistics Dashboard", style={'textAlign': 'center'}),  # May include style for title
    html.Div([  # TASK 2.2: Add two dropdown menus
        html.Label("Select Statistics:"),
        dcc.Dropdown(
            id='dropdown-statistics',
            options=[
                {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
            ],
            value='Select Statistics',
            placeholder='Select a report type',
            style={'width': '80%', 'padding': '3px', 'fontSize': '20px', 'textAlignLast': 'center'},
        )
    ]),
    html.Div([
        dcc.Dropdown(
            id='select-year',
            options=[{'label': i, 'value': i} for i in year_list],
            placeholder='Select Year',
            style={'width': '80%', 'padding': '3px', 'fontSize': '20px', 'textAlignLast': 'center'},
        )
    ]),
    html.Div([  # TASK 2.3: Add a division for output display
        html.Div(id='output-container', className='chart-grid', style={'flex': 1}),
    ])
])


In [17]:
# Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics', component_property='value')
)
def update_input_container(selected_statistic):
    return False if selected_statistic == 'Yearly Statistics' else True

# Define the callback function to update the output container based on the selected statistics
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='select-year', component_property='value'),
     Input(component_id='dropdown-statistics', component_property='value')]
)
def update_output_container(selected_year, selected_statistic):
    if selected_statistic == 'Recession Period Statistics':
        # Filter the data for recession periods based on selected year if available
        if selected_year is not None:
            recession_data = data[(data['Year'] == selected_year) & (data['Recession'] == 1)]
        else:
            recession_data = data[data['Recession'] == 1]
        # Create a table to display filtered data
        return html.Table(
            # Assuming 'recession_data' is a DataFrame containing filtered data
            [html.Tr([html.Th(col) for col in recession_data.columns])] +
            [html.Tr([html.Td(recession_data.iloc[i][col]) for col in recession_data.columns])
             for i in range(len(recession_data))]
        )
    else:
        return html.Div("No data to display")  # Display a message if no data is available


In [18]:
# task 2.5
# Assuming the function where this code resides has access to recession_data
def generate_recession_charts(recession_data):
    # Plot 1: Automobile sales fluctuate over Recession Period (year wise) using line chart
    yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
    R_chart1 = dcc.Graph(
        figure=px.line(yearly_rec, 
                       x='Year',
                       y='Automobile_Sales',
                       title="Average Automobile Sales Fluctuation over Recession Period"))

    # Plot 2: Calculate the average number of vehicles sold by vehicle type and represent as a Bar chart
    average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
    R_chart2 = dcc.Graph(
        figure=px.bar(average_sales,
                      x='Vehicle_Type', 
                      y='Automobile_Sales', 
                      title="Average Number of Vehicles Sold by Vehicle Type"))

    # Plot 3: Pie chart for total expenditure share by vehicle type during recessions
    exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
    R_chart3 = dcc.Graph(
        figure=px.pie(exp_rec,
                      values='Advertising_Expenditure',
                      names='Vehicle_Type',
                      title="Total Expenditure Share by Vehicle Type during Recessions"))

    # Plot 4: Develop a Bar chart for the effect of unemployment rate on vehicle type and sales
    unemployment_rate = recession_data.groupby('Vehicle_Type')['Unemployment_Rate'].mean().reset_index()
    R_chart4 = dcc.Graph(
        figure=px.bar(unemployment_rate, 
                      x='Vehicle_Type', 
                      y='Unemployment_Rate', 
                      title='Unemployment Rate by Vehicle Type'))

    # Returning the charts
    return html.Div([
        html.Div(className='chart-item', children=[html.Div(children=R_chart1), html.Div(children=R_chart2)], style={'display':'flex'}),
        html.Div(className='chart-item', children=[html.Div(children=R_chart3), html.Div(children=R_chart4)], style={'display':'flex'})
    ])


In [19]:
def some_function():
    # Task 2.6
    # TASK 2.6: Returning the graphs for displaying Yearly data
    if condition:  # Replace 'condition' with the appropriate condition for your function
        return [
            html.Div(className='chart-item', children=[
                html.Div(Y_chart1),
                html.Div(Y_chart2)
            ], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[
                html.Div(Y_chart3),
                html.Div(Y_chart4)
            ], style={'display': 'flex'})
        ]
    else:
        return None

In [20]:
# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)